In [ ]:
%load_ext autoreload
%autoreload 2

# log

> customized logging service for the system programming 


In [ ]:
#| default_exp system.log

In [ ]:
#| export
import inspect
from typing import Tuple

In [ ]:
#| export
# Logging Service Initialization
import logging
from zoneinfo import ZoneInfo

In [ ]:
#| export
# system imports
import os
from pathlib import Path, PurePosixPath
from logging.handlers import SocketHandler

In [ ]:
#| export
from pythonjsonlogger import jsonlogger
import pandas as pd

In [ ]:
#| export

# logging.basicConfig(level=logging.DEBUG, format=fmt)
mpl_logger = logging.getLogger("matplotlib.font_manager")
mpl_logger.disabled = True

In [ ]:
#| export
def set_root_logger(
    name: str,  # name of the root logger
    data_root: Path,  # root path of the data
    agent: str,  # agent name, ie. "DDPG"/"RDPG"
    tz: ZoneInfo,  # time zone of the logging
    truck: str,  # truck name, ie. "VB7"
    driver: str,  # driver name, ie. "wang-cheng"
) -> Tuple[logging.Logger, dict]:  # return the logger and the dict_logger
    """
    Set the root logger for the system
    """

    logger = logging.getLogger(name)
    logger.propagate = False

    dict_logger = {"user": inspect.currentframe().f_code.co_name}  # type: ignore
    log_root = data_root / "py_logs"
    try:
        os.makedirs(log_root)
    except FileExistsError:
        print("User folder exists, just resume!")

    log_file_name = log_root.joinpath(
        "eos-rt-"
        + str(agent)
        + "-"
        + truck
        + "-"
        + driver
        + "-"
        + pd.Timestamp.now(tz).isoformat()  # .replace(":", "-")
        + ".log"
    )
    fmt = "%(asctime)s-%(name)s-%(levelname)s-%(module)s-%(threadName)s-%(funcName)s)-%(lineno)d): %(message)s"
    formatter = logging.Formatter(fmt)
    logging.basicConfig(
        format=fmt,
        datefmt="%Y-%m-%dT%H:%M:%S.%f",
    )
    json_file_formatter = jsonlogger.JsonFormatter(
        "%(created)f %(asctime)s %(name)s "
        "%(levelname)s %(module)s %(threadName)s %(funcName)s) %(lineno)d) %(message)s"
    )

    file_handler = logging.FileHandler(log_file_name)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(json_file_formatter)
    # str_file_name = PurePosixPath(log_file_name).stem + ".json"
    str_file_name = log_root.joinpath(PurePosixPath(log_file_name).stem + ".json")
    str_handler = logging.FileHandler(str_file_name, mode="a")
    str_handler.setLevel(logging.DEBUG)
    str_handler.setFormatter(json_file_formatter)

    char_handler = logging.StreamHandler()
    char_handler.setLevel(logging.DEBUG)
    char_handler.setFormatter(formatter)
    #  Cutelog socket
    socket_handler = SocketHandler("127.0.0.1", 19996)
    socket_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(str_handler)
    logger.addHandler(char_handler)
    logger.addHandler(socket_handler)

    logger.setLevel(logging.DEBUG)

    # tf_logger = tf.set_root_logger()
    # tf_logger.addHandler(file_handler)
    # tf_logger.addHandler(char_handler)
    # tf_logger.addHandler(socket_handler)
    # tf_logger.addHandler(str_handler)
    return logger, dict_logger

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
